<a href="https://www.kaggle.com/code/mingtr/python-cheatsheet-for-eda?scriptVersionId=176849521" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Purpose of this notebook

- Provide a cheat sheet of EDA.
- We will not focus on the profound knowledge of EDA here. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from scipy import stats
! pip install yfinance
import yfinance 

# to ignore some warnings in python
import warnings 
warnings.filterwarnings('ignore')

# display all the columns and rows 
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', None),

# set figure size for single graphs 
# plt.rcParams['figure.figsize'] = [15, 6]

<a id = "table-of-content"></a>
# Table of content

- [ Import Data and Basic Examination](#1)
- [ Simple plots with df.plot()](#2)
- [ Relationships among variables via plots](#3)
- [ Maths](#4)
- [ Convert df, list, set, zip and mapping](#5)
- [ Export to excel or csv, modify available]  file(#6)


<a id = "1"></a>
# Import Data and Basic Examination

In [ ]:
df1 = pd.read_csv("/kaggle/input/car-prices-market/used_car_prices.csv")
df2 = pd.read_csv("/kaggle/input/car-prices-market/new_cars_prices.csv")
# df2 = pd.read_csv("/kaggle/input/car-prices-market/new_cars_prices.csv", sheet_name='new_cars_prices', skiprows=None)

In [ ]:
df = yfinance.download(tickers= ['GC=F', 'ADBE', 'AMZN', 'NVDA', 'PKX', 'WMT'], 
                         start="2020-01-30", end="2022-01-30", 
                         group_by='column')['Adj Close']

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df.head()

## Drop multiple index or headers

In [ ]:
#df = df.droplevel(0, axis=1).rename_axis(index=(None, None), columns=None)

## Overview

In [ ]:
df1.info()

In [ ]:
# describe with some aspects only, like max values
df1.describe().loc[['count','top']]

In [ ]:
df1.shape

In [ ]:
df1.dtypes

## Unique values, Value counts

In [ ]:
# Count number of distinct elements by row
df1.nunique(axis=0, dropna=True)

In [ ]:
# Count number of distinct elements by column
df1.nunique(axis=1)

In [ ]:
df1['Car Model'].unique()

In [ ]:
df1["Car Model"].value_counts()

<a id = "3"></a>
## Column names, index changing

In [ ]:
df1.columns

In [ ]:
# change the name of columns
df1.columns = ['web_scraper_order', 'Car Model', 'Month/Year', 'Average price', 'Minimum price', 'Maximum price']

In [ ]:
# Another way
df1.rename(columns={"web-scraper-order" : "web_scraper_order",
                        "Car Model"         : "car_model",
                        "Month/Year"        : "month_year",
                        "Average price"     : "average_price",
                        "Minimum price"     : "minimum_price",
                        "Maximum price"     :  "maximum_price"}, 
           inplace=True, errors='ignore') # can set raise error if needed

In [ ]:
df.reset_index(inplace = True, 
               #drop = True 
              )

## Null values

In [ ]:
df1.isnull().values.any()

In [ ]:
df1.isnull().sum().sort_values(ascending=False)

In [ ]:
# print out the rows with null values
df_null_row = df1[df1.isna().any(axis=1)]
df_null_row.head() 

In [ ]:
df1.dropna(inplace=True)

In [ ]:
# Drop null values with conditions
# Drop the rows where at least one element is missing
# df1.dropna(,inplace=True)

In [ ]:
#Define in which columns to look for missing values.
#df.dropna(subset=['name', 'toy'])

In [ ]:
# Drop the columns where at least one element is missing.
# df.dropna(axis='columns')

In [ ]:
#Drop the rows where all elements are missing.
#df.dropna(how='all')

In [ ]:
# drop null values based on a threshold
# df.dropna(thresh = len(df) * 0.2, axis = 1, inplace = True)

## Drop rows based on values

In [ ]:
# df2[df2['PARTY_TYPE'].isin(['OTC', 'CSA'])]

## Date and time

In [ ]:
# fill all rows in a column with the same values
df['today'] = '2023-05-02'

In [ ]:
import datetime
datetime.date(2022, 12, 25)

In [ ]:
from datetime import date
d = date(2022, 12, 25)
print(d)

In [ ]:
from datetime import datetime
# convert 
df['today'] = pd.to_datetime(df['today'])

# Calculate days between 2 columns of datetime 
df['days_count'] = (pd.Timestamp('today') - df['Date']).dt.days
df.head()

In [ ]:
# get the current date and time
datetime.now()

In [ ]:
todays_date = date.today()
print("Today's date =", todays_date)

In [ ]:
# date object of today's date
today = date.today() 

print("Current year:", today.year)
print("Current month:", today.month)
print("Current day:", today.day)

## String, text

In [ ]:
list_col =['average_price','minimum_price','maximum_price']
for i in list_col:
    df1[i] = df1[i].str.strip('EGP')
    df1[i] = df1[i].str.replace(',', '')
    df1[i] = df1[i].str.strip(' ')
    df1[i] = df1[i].astype('int')

In [ ]:
df1.dtypes

In [ ]:
# other
t = '2023-03'
t.split('-')[0]

In [ ]:
list_year = []
for i in df1['month_year']:
    a = i.split('-')[0]
    list_year.append(a)
df1['year'] = list_year

In [ ]:
a_string = 'This is a string'
first_four_letters = a_string[:4]
first_four_letters

In [ ]:
df1.head()

## Filter lines that contain an indicated value

In [ ]:
df1[df1['car_model'].str.contains("Seat Ateca 2021")]

## Fill, slice values

In [ ]:
df1.columns

In [ ]:
df1[['car_model','average_price']].head()

In [ ]:
df1.loc[6:8]

## Loop, concat with loop, lamba

In [ ]:
# Split Car Full Data into Brand, Model, And Year of The Model

model = df1['car_model'].apply(lambda x: ''.join((x.split()[1],' ',x.split()[2])) if len(x.split()) >3 else x.split()[1])
year = df1['car_model'].apply(lambda x: x.split()[-1])
brand = df1['car_model'].apply(lambda x: x.split()[0])

df1['brand'], df1['model'], df1['model_year'] = brand,model,year

In [ ]:
# total rows of 2 dataframes
df1.shape[0] + df2.shape[0]

In [ ]:
# total columns of 2 dataframes
df1.shape[1] + df2.shape[1]

In [ ]:
# Take a small part of dataframe to test
df1_test = df1.head(3)[['web_scraper_order','car_model']]
df2_test = df2.head(3)[['web-scraper-order','Car Model']]

In [ ]:
df1_test

In [ ]:
df2_test

In [ ]:
# concat 2 dataframes with the same number of columns and same names
pd.concat([df1_test,df2_test.rename(columns= {'web-scraper-order':'web_scraper_order',
                                             'Car Model':'car_model'})], ignore_index=True)

In [ ]:
# merge data using a column
# add a random columns for testing
df1_test['color'] = ['red', 'blue', 'green']
df2_test['color'] = ['blue', 'red', 'red']
print(df1_test)
print(df2_test)
df1_test.merge(df2_test, how='inner', on='color')

In [ ]:
df1_test.merge(df2_test, how='outer', on='color')

<a id = "2"></a>
# Simple plots with df.plot()

## Line graph

In [ ]:
df['AMZN'].plot().set_ylabel("Amazon prices")

## Histogram and pie chart

In [ ]:
df['AMZN'].plot(kind='hist', bins=20)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 6))

df1['average_price'].hist(bins=10,ax=axs[0])
axs[0].set_title('average_price', fontsize=16)
axs[0].set_ylabel("Frequency")
axs[0].set_xlabel("EGP")

df1['year'].value_counts().plot.pie(ax=axs[1], autopct='%.2f%%')
axs[1].set_title('Year#', fontsize=16)
plt.show()

## Bar chart

In [ ]:
# in percentage
df1['year'].value_counts(normalize=True).mul(100).plot.bar()

## Boxplot

In [ ]:
df1.boxplot(column=['minimum_price', 'maximum_price'], grid=True,)

<a id = "3"></a>
# Relationships among variables via plots

## Univariate

In [ ]:
df.cov()
df.var()
df.std()

In [ ]:
df.head()

In [ ]:
#skewness and kurtosis
print("Skewness: %f" % df['ADBE'].skew())
print("Kurtosis: %f" % df['ADBE'].kurt())

In [ ]:
#histogram
sns.distplot(df['AMZN'])

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(20, 5))
sns.distplot(df['ADBE'], ax=axs[0])
sns.distplot(df['AMZN'], ax=axs[1])
sns.distplot(df['NVDA'], ax=axs[2]).set_title("Distribution of each returns")
sns.distplot(df['PKX'], ax=axs[3])
sns.distplot(df['WMT'], ax=axs[4])
plt.show()

In [ ]:
# probability plot
res = stats.probplot(df['AMZN'], plot=plt)

## Bivariate
### Numeric - Numeric

- Correlation Matrix
- Scatterplot

### Correlation Matrix

In [ ]:
# correlation matrix
cm = df1.corr()
sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10})

### Scatterplot

In [ ]:
sns.pairplot(data=df1)

## Category - Category dimentions

In [ ]:
pd.crosstab(df1['brand'], df1['year']).plot(kind='bar', stacked=True)
plt.title('Year vs Model', fontsize=16)
plt.ylabel('Frequency')
plt.show()

## Category - Numeric dimentions

In [ ]:
sns.boxplot(x="year", y="minimum_price", data=df1, 
            palette="Accent", #color='g'
           )

<a id = "4"></a>
# Maths

## For trading

In [ ]:
# log return
df['ADBE_ret_log'] = np.log(df.ADBE/df.ADBE.shift(1))

In [ ]:
# pct return
df['ADBE_ret_pct'] = df['ADBE'] / df['ADBE'].shift(1)
df.head()

In [ ]:
# annualized
df.cov()*252
df.std()*np.sqrt(252)

In [ ]:
# updating

## Combinations

In [ ]:
from itertools import combinations, combinations_with_replacement
 
# Get all combinations of [1, 2, 3]
# and length 2
comb = combinations(['Beer','Whiskey','CubaLibre'], 3)
 
# Print the obtained combinations
for i in list(comb):
    print (i)

In [ ]:
comb = combinations_with_replacement(['Beer','Whiskey','CubaLibre'], 3)
 
# Print the obtained combinations
for i in list(comb):
    print (i)

## Permutation

In [ ]:
from itertools import permutations

# Define a list of elements
elements = ['A', 'B', 'C']

# Calculate permutations without repetition
perms = list(permutations(elements, 2))  # Generate permutations of length 2

# Print permutations
print("Permutations without repetition:", perms)


In [ ]:
from itertools import product

# Define a list of elements
elements = ['A', 'B', 'C']

# Calculate permutations with repetition
perms_with_repetition = list(product(elements, repeat=2))  # Generate permutations with repetition of length 2

# Print permutations with repetition
print("Permutations with repetition:", perms_with_repetition)


<a id = "5"></a>
# Convert df, list, set, zip and mapping

In [ ]:
x = df1['car_model'].unique().tolist()
#x

In [ ]:
# zip and map
y = ['Category A', 'Category B', 'Category C']
z = dict(zip(x,y))
df1['Category'] = df1['car_model'].map(z)

In [ ]:
df1[:15]

In [ ]:
s = pd.Series(['1.0', '2', -3])
pd.to_numeric(s, downcast='float')

<a id = "6"></a>
# Export to excel or csv, modify available file

In [ ]:
#df1.to_csv("old_cars_df.csv", index=False)
#df2.to_csv("new_cars_df.xlsx", sheet_name='Details',index=False)

In [ ]:
#with pd.ExcelWriter('new_cars_df.xlsx', engine='openpyxl', mode='a') as writer:  
#    df2.to_excel(writer, sheet_name='Customer2')